In [1]:
from pathlib import Path
from functools import partial
import pandas as pd
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

import numpy as np
import mlflow
from scipy.ndimage import zoom

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
from unxpass.databases import SQLiteDatabase
from unxpass.datasets import PassesDataset, CompletedPassesDataset, FailedPassesDataset
from unxpass.components import pass_selection, pass_value, pass_success, pass_value_custom
from unxpass.components.utils import load_model
from unxpass.visualization import plot_action
from unxpass.ratings_custom import LocationPredictions

from matplotlib.backends.backend_pdf import PdfPages

from notebooks import playVisualizers
model_pass_value = pass_value.SoccerMapComponent(
    model=mlflow.pytorch.load_model(
        'runs:/20e7d3695d7049d0a513922d32b44a11/model', map_location='cpu'
        #'runs:/788ec5a232af46e59ac984d50ecfc1d5/model', map_location='cpu'
    ), offensive = False
)
DATA_DIR = Path("../stores/")
dbpath = "/home/lz80/rdf/sp161/shared/soccer-decision-making/Bundesliga/buli_all.sql"
feat_path = "/home/lz80/rdf/sp161/shared/soccer-decision-making/Bundesliga/all_features_fail"
db = SQLiteDatabase(dbpath)
dataset_test = partial(PassesDataset, path = feat_path)
#surfaces = model_pass_value.predict_surface(dataset_test, db = db, model_name = "val", game_id = game_id)

In [ ]:
a1 = [{"actor":False,"keeper":False,"teammate":False,"x":51.89714748750001,"y":25.764023090000002},{"actor":False,"keeper":False,"teammate":False,"x":60.595448024999996,"y":39.9492384},{"actor":False,"keeper":False,"teammate":False,"x":71.0161843125,"y":38.480520170000005},{"actor":False,"keeper":False,"teammate":False,"x":59.6194011,"y":10.007837014999993},{"actor":False,"keeper":False,"teammate":False,"x":52.1268055875,"y":41.873445195},{"actor":False,"keeper":False,"teammate":False,"x":70.93963161250001,"y":22.445463545000003},{"actor":False,"keeper":True,"teammate":False,"x":95.5130483125,"y":33.860564724999996},{"actor":False,"keeper":False,"teammate":False,"x":71.2649805875,"y":33.386484790000004},{"actor":False,"keeper":False,"teammate":False,"x":70.6716971625,"y":13.828363549999999},{"actor":False,"keeper":False,"teammate":False,"x":61.858567575,"y":20.790831615000002},{"actor":False,"keeper":False,"teammate":False,"x":65.55223535,"y":28.180901189999993},{"actor":False,"keeper":True,"teammate":True,"x":20.500971399999997,"y":33.739720819999995},{"actor":False,"keeper":False,"teammate":True,"x":53.533461450000004,"y":3.6960669000000053},{"actor":False,"keeper":False,"teammate":True,"x":44.4715355875,"y":36.15659892},{"actor":False,"keeper":False,"teammate":True,"x":42.8543598,"y":26.284581450000005},{"actor":False,"keeper":False,"teammate":True,"x":52.949747112500006,"y":14.767227734999992},{"actor":True,"keeper":False,"teammate":True,"x":52.4904309125,"y":33.209866774999995},{"actor":False,"keeper":False,"teammate":True,"x":36.5483311375,"y":37.430107765},{"actor":False,"keeper":False,"teammate":True,"x":37.6583452875,"y":23.988547254999993},{"actor":False,"keeper":False,"teammate":True,"x":42.9404815875,"y":40.795145735000006},{"actor":False,"keeper":False,"teammate":True,"x":40.002771725,"y":14.423287389999992},{"actor":False,"keeper":False,"teammate":True,"x":52.99759255,"y":22.473350599999996}]
a0 = [{"actor":True,"keeper":False,"teammate":True,"x":63.561865149999996,"y":7.070400554999992},{"actor":False,"keeper":False,"teammate":True,"x":66.11681151249999,"y":30.170177780000003},{"actor":False,"keeper":False,"teammate":True,"x":65.5426662625,"y":45.95425091},{"actor":False,"keeper":False,"teammate":True,"x":62.308314687499994,"y":3.3056481299999945},{"actor":False,"keeper":False,"teammate":True,"x":74.9395101875,"y":23.337849305},{"actor":False,"keeper":False,"teammate":True,"x":45.1987862375,"y":13.177665599999997},{"actor":False,"keeper":True,"teammate":True,"x":19.9363952375,"y":30.504822439999998},{"actor":False,"keeper":False,"teammate":True,"x":52.308618249999995,"y":25.196986305000003},{"actor":False,"keeper":False,"teammate":True,"x":51.6866275625,"y":4.281695055},{"actor":False,"keeper":False,"teammate":True,"x":70.20281187500001,"y":10.472621265},{"actor":False,"keeper":False,"teammate":True,"x":56.5285858375,"y":13.177665599999997},{"actor":False,"keeper":True,"teammate":False,"x":94.64226135,"y":30.281726},{"actor":False,"keeper":False,"teammate":False,"x":51.753611174999996,"y":11.606694834999999},{"actor":False,"keeper":False,"teammate":False,"x":67.140703875,"y":15.064689655000002},{"actor":False,"keeper":False,"teammate":False,"x":64.116872225,"y":6.930965279999995},{"actor":False,"keeper":False,"teammate":False,"x":60.327513575,"y":5.136898074999991},{"actor":False,"keeper":False,"teammate":False,"x":61.94468936250001,"y":21.785469909999996},{"actor":False,"keeper":False,"teammate":False,"x":76.1643533875,"y":23.01250033},{"actor":False,"keeper":False,"teammate":False,"x":73.6285452,"y":11.114023529999997},{"actor":False,"keeper":False,"teammate":False,"x":73.5424234125,"y":29.417227294999996},{"actor":False,"keeper":False,"teammate":False,"x":64.844122875,"y":5.861961505000004},{"actor":False,"keeper":False,"teammate":False,"x":61.360975024999995,"y":9.292069269999999}]
a0_col = pd.Series([a0] * 1000)
a1_col = pd.Series([a1] * 1000)
time_col = pd.Series(np.random.uniform(0.1, 0.5, 1000))
def get_speeds(a0, a1, timediff):
    output = []
    a0locs = [[p['x'], p['y']] for p in a0]
    for player in a1:
        playerdict = {}
        dists = min([np.linalg.norm(np.array(p) - np.array([player['x'], player['y']])) for p in a0locs])
        speed = dists / timediff
        playerdict = {
            "actor":player['actor'],
            "keeper":player['keeper'],
            "teammate":player['teammate'],
            "x":player['x'],
            "y":player['y'],
            "speed":speed
        }
        output.append(playerdict)
    return output
testdf = pd.DataFrame(data = {"a0": a0_col, "a1": a1_col, "timediff": time_col})
testdf.index = a0.index
#
testdf.apply(lambda x: get_speeds(*x), axis=1).rename({0: "player_speeds_a01"})

In [25]:
#team with possession should be going right
from unxpass import load_xml

def getFlips(game_id):
    events = load_xml.load_event(f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Bundesliga/event_data_all/{game_id}.xml")
    event_csv = load_xml.load_csv_event(f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Bundesliga/KPI_Merged_all/KPI_MGD_{game_id}.csv")
    events['TeamLeft'] = events['TeamLeft'].fillna(method='ffill')
    events['TeamRight'] = events['TeamRight'].fillna(method='ffill')
    #direction of play to right
    #if team in possession is team left - all good
    #if team in possession is team right - need to flip
    return events[events['Team'] == events['TeamRight']]['EventId']
def add_before_buli(game_id, timeback = 0.04):
    flips = getFlips(game_id)
    #print(flips)
    print("Loading Data...")
    eventpath = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Bundesliga/converted_data/events/{game_id}.json"
    threesixty = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Bundesliga/converted_data/three-sixty/{game_id}.json"
    tracking = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Bundesliga/zipped_tracking/zip_output/{game_id}.xml"
    kpievent = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Bundesliga/KPI_Merged_all/KPI_MGD_{game_id}.csv"
    eventdf = pd.read_json(eventpath)
    threesixty_df = pd.read_json(threesixty)
    trackingdf = load_xml.load_tracking(tracking)
    kpidf = load_xml.load_csv_event(kpievent)
    outeventpath = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Bundesliga/converted_data_b/events/{game_id}.json"
    outthreesixty = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Bundesliga/converted_data_b/three-sixty/{game_id}.json"
    def frametodict(group):
        """
        Takes a group of tracking data and returns a list of dictionaries
        """
        noball = group[group['TeamId'] != "BALL"]
        noball["X_translated"] = 120 - (1.09361 * noball["X"].astype(float) + 60)
        noball["Y_translated"] = 80 - (-1.09361 * noball["Y"].astype(float) + 40)
        noball['360frame'] = noball.apply(lambda x: {
            'location': [x['X_translated'], x['Y_translated']],
            'actor': False,
            'keeper': False,
            'teammate': False
        }, axis=1)
        return noball['360frame'].to_list()
    times = kpidf[['EVENT_ID', 'TRACKING_TIME']]
    timedf = pd.merge(eventdf, times, left_on = "id", right_on = "EVENT_ID", how = 'left').drop(columns = ["EVENT_ID"])
    #framediff = pd.to_datetime(trackingdf['T'])[1] - pd.to_datetime(trackingdf['T'])[0]
    trackingdf['T'] = pd.to_datetime(trackingdf['T'])
    timedf['TRACKING_TIME'] = pd.to_datetime(timedf['TRACKING_TIME']).dt.tz_localize('UTC') - pd.Timedelta(hours=2)
    framediff = pd.Timedelta(seconds = 0.04)
    def getframes(row, timeback):
        """
        Creates s-360 tracking equivalents for dummy events prior to event of interest
        """
        id = row['id']
        ogtime = row['timestamp']
        time = row['TRACKING_TIME']
        firsttime = time - pd.Timedelta(seconds = timeback)
        
        prior = trackingdf[trackingdf['T'].between(firsttime, time)]
        frames = prior.groupby('N').apply(frametodict)
        #print(id)
        if(str(id) in flips.unique()): #kinda janky but whatevs
            #print("Flipping", end = "/r")
            for frame in range(len(frames)):
                frames[frame] = [{'teammate': player['teammate'],
                    'actor':player['actor'],
                    'keeper':player['keeper'],
                    'location': [120 - player['location'][0], 80 - player['location'][1]]} for player in test_list]
        ids = [f"{id}-b-{i}" for i in range(len(frames), 0, -1)]
        visible_area = [[0, 0, 120, 0, 120, 80, 0, 80, 0, 0]] * len(frames)
        #print(ids)
        threesixty = pd.DataFrame({"event_uuid": ids, "visible_area":visible_area, "freeze_frame":frames})
        added_events = pd.DataFrame([row] * len(threesixty))
        added_events['id'] = ids
        added_events['timestamp'] = [ogtime - i for i in [j * framediff for j in range(1, len(frames) + 1)]]
        added_events['index'] = [row['index'] - 1 + i * (1 / (len(frames) + 1)) for i in range(1, len(frames) + 1)]
        #.to_list()
        return threesixty, added_events
    #getframes(timedf.loc[2])
    print("Adding Dummy Events")
    passes = timedf[timedf['type'] == {'id': 30, 'name': 'Pass'}]
    newevents = [eventdf]
    newthreesixty = [threesixty_df]
    iter = 1
    for idx, row in passes.iterrows():
        print(f"{iter} / {passes.shape[0]}", end='\r')
        threesixty, added_events = getframes(row, timeback = timeback)
        newthreesixty.append(threesixty)
        newevents.append(added_events)
        iter+= 1
    addedevents = pd.concat(newevents)
    addedthreesixty = pd.concat(newthreesixty)
    addedevents.sort_values(by = "index", inplace = True)
    addedthreesixty.to_json(outthreesixty, orient = "records", lines = True)
    addedevents.to_json(outeventpath, orient = "records", lines = True)
add_before_buli(game_id = "DFL-MAT-J03YDU")

Loading Data...
Adding Dummy Events


In [ ]:
matches_map = {}
from os import listdir
game_dir = "/home/lz80/rdf/sp161/shared/soccer-decision-making/allHawkEye/"
games = pd.read_json("/home/lz80/rdf/sp161/shared/soccer-decision-making/Hawkeye_AllGames/matches/53/106.json", convert_dates = False)#directory of statsbomb matches data
games['home_team'] = games.apply(lambda d: d['home_team']['home_team_name'], axis = 1).str.replace("Women's", "").str.replace("WNT", "").str.strip()
games['away_team'] = games.apply(lambda d: d['away_team']['away_team_name'], axis = 1).str.replace("Women's", "").str.replace("WNT", "").str.strip()
dirfiles = [f for f in listdir(game_dir) if not f.startswith('.')]
for game in dirfiles:
    print(game)
    home_team = game.split('_')[1]
    away_team = game.split('_')[2]
    #print(games['away_team'])
    print(home_team, away_team)
    game_id = games[(games['home_team'] == home_team) & (games['away_team'] == away_team)].reset_index().loc[0]['match_id']
    matches_map[game] = int(game_id)
print(matches_map)
import json 
outpath = "/home/lz80/rdf/sp161/shared/soccer-decision-making/hawkeye_to_sb.json"
with open(outpath, "w") as f:
    json.dump(matches_map, f)

In [ ]:
#adding events prior for hawkeye
def hawkeyeGetBefore(row):
    with open("/home/lz80/rdf/sp161/shared/soccer-decision-making/hawkeye_to_sb.json", 'r') as file:
        hawkeye_to_sb = json.load(file)
    sb_to_hawkeye = {v: k for k, v in hawkeye_to_sb.items()}
    match_id = row['match_id']
    hawkeye_id = sb_to_hawkeye[match_id]

    id = row['id']
    time = row['BallReceipt']
    sb_event = pd.read_json(f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Hawkeye_AllGames/events/{match_id}.json")
    sb_tracking = pd.read_json(f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Hawkeye_AllGames/three-sixty/{match_id}.json")
    sb_matches = pd.read_json("/home/lz80/rdf/sp161/shared/soccer-decision-making/Hawkeye_AllGames/matches/53/106.json")
    tracking = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/allHawkeye/{hawkeye_id}/scrubbed.samples.centroids"
    out_sb_event = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Hawkeye_b/events/{match_id}.json"
    out_three_sixty = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Hawkeye_b/three-sixty/{match_id}.json"
    file_path_begin = "2021_18_2032206"
    import conversions 
    #)
    timeback = 0.1
    sequences.columns
    period = row['period']
    minute = int(time / 60) + 1
    second = int(time) % 60
    second_range = (second - timeback, second)
    team = row['team_id']
    actor = row['player_id']
    uefa_map = {}
    #goalkeepers = []
    file_path = f"{tracking}/{file_path_begin}_{str(period)}_{str(minute)}.football.samples.centroids"
    def getGksTM(game_id):
        lineups = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/womens_euro_receipts/lineups/{game_id}.json"
        lineup_df = pd.read_json(lineups, convert_dates = False)
        team_1 = lineup_df['team_id'].loc[0]
        team_2 = lineup_df['team_id'].loc[1]
        team_1_dict = lineup_df['lineup'].loc[0]
        team_2_dict = lineup_df['lineup'].loc[1]
        team_1_lineup = [player_dict['player_id'] for player_dict in team_1_dict]
        team_2_lineup = [player_dict['player_id'] for player_dict in team_2_dict]
        team_map = {team_1 : team_1_lineup, team_2 : team_2_lineup}
        player_to_team = {player_id: team_id for team_id, players in team_map.items() for player_id in players}
        pos_dict = {player['player_id']: player['positions'][0]['position'] for player in team_1_dict if len(player['positions']) > 0}
        team_2_pos_dict = {player['player_id']: player['positions'][0]['position'] for player in team_2_dict if len(player['positions']) > 0}
        pos_dict.update(team_2_pos_dict)
        goalkeepers = [key for (key,value) in pos_dict.items() if value == "Goalkeeper"]
        return player_to_team, goalkeepers
    player_to_team, goalkeepers = getGksTM(match_id)
    all_locs = []
    loc1 = conversions.read_Hawkeye_player_loc(file_path, period, minute, second_range, team,actor, id, player_to_team, goalkeepers)
    all_locs.append(loc1)
    if(second - timeback < 0):
        #if the time is negative, we need to get the last frame of the previous minute
        #print("Getting last frame of previous minute")
        file_path = f"{tracking}/{file_path_begin}_{str(period)}_{str(minute - 1)}.football.samples.centroids"
        second_range = (59 + second - timeback, 59)
        loc2 = conversions.read_Hawkeye_player_loc(file_path, period, minute - 1, second_range, team,actor, id, player_to_team, goalkeepers)
        all_locs.append(loc2)
    addedtracking = pd.concat(all_locs)
    addedtracking['event_uuid'] = [f"{id}-b{i}" for i in range(len(addedtracking), 0, -1)]
    framesback = addedtracking.shape[0]
    #conversions
    def add_events(id, framesback):
        #.04 seconds per frame
        sb_row = sb_event[sb_event['id'] == id].reset_index().loc[0]
        ogtime = sb_row['timestamp']
        framediff = pd.Timedelta(seconds = 0.04)
        added_events = pd.DataFrame([sb_row] * framesback)
        added_events['id'] = [f"{id}-b{i}" for i in range(framesback, 0, -1)]
        added_events['timestamp'] = [ogtime - i for i in [j * framediff for j in range(1, framesback + 1)]]
        added_events['index'] = [row['index'] - 1 + i * (1 / (framesback + 1)) for i in range(1, framesback + 1)]
        return added_events
    added_events = add_events(id, framesback)
    eventsadded = pd.concat([sb_event, added_events])
    threesixtyadded = pd.concat([sb_tracking, addedtracking])
    threesixtyadded.to_json(out_three_sixty, orient = "records", lines = True)
    eventsadded.to_json(out_sb_event, orient = "records", lines = True)
    return added_events, addedtracking
import json
sequences = pd.read_csv("/home/lz80/un-xPass/unxpass/steffen/sequences_new.csv")
row = sequences.loc[0]
added_events, added_tracking = hawkeyeGetBefore(row)

In [46]:
player_df_all = pd.read_json(file_path, lines = True, orient = 'columns')
player_dict = player_df_all['samples'].loc[0]['people']
player_df = pd.DataFrame(player_dict)
times = player_df['centroid'].apply(lambda x: x[0]['time'])
times.unique()[0] - times.unique()[1]

-0.039998000000000006

In [ ]:
game_id = 'DFL-MAT-J03YDU'
labs = model_pass_value.initialize_dataset(dataset_test, model_name = "val").labels
fails = labs[labs['concedes_xg'] > 0].index
#db.actions(game_id = game_id).loc[fails]
gameidx = [idx for idx in fails if idx[0] == game_id]
eventids = db.actions(game_id = game_id).loc[gameidx]['original_event_id']


In [48]:
from unxpass.load_xml import load_tracking, load_csv_event
from unxpass.visualizers_made import get_animation_from_raw
def get_trackingevent(game_id):
    trackingpath = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Bundesliga/zipped_tracking/zip_output/{game_id}.xml"
    eventpath = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/Bundesliga/KPI_Merged_all/KPI_MGD_{game_id}.csv"
    eventdf = load_csv_event(eventpath)
    trackingdf = load_tracking(trackingpath)
    return trackingdf, eventdf

trackingdf, eventdf = get_trackingevent(game_id)

In [ ]:
for event_id in eventids:
    print(event_id)

In [ ]:
itera = 1
framerate = 5
eventid = eventids[itera]
test = get_animation_from_raw(float(eventid), framerate, eventdf, trackingdf, frameskip = 20, show = False, add_frames = 600)
#eventdf[eventdf["EVENT_ID"] == float(eventid)]
#eventid


In [ ]:
test.save("file.gif")